# Implementation of a pure (non-hybrid) Quantum Neural Network with entangled states
## The notebook below implements a multi-layered perceptron based Neural Network with hidden layers implemented replicated by varational circuits
    - The implementation is a muticlass classifier that tries to predict the 3 output classes
    - This is a novel algorithm that is not implemented on the pennylane tutorial website
    - The dataset used is IRIS flower classification dataset, with 5 input features and 3 output classes
    - The performance is evaluated on the test dataset with train-test split of 75%:25%import pennylane as qml
from pennylane import numpy as np

In [10]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer, AdamOptimizer
from pennylane.init import strong_ent_layers_uniform
from pennylane.templates.layers import StronglyEntanglingLayers
from pennylane.templates import AmplitudeEmbedding


In [19]:
n_qubits = 3 # for representing three output classes
num_layers = 3 # number of hidden layers of the quantum neural network
q_depth = num_layers # Circuit depth which is also equal to num_layers
l_r = 0.1 # Learning rate for the optimizer
num_epochs = 100 # number of training epochs

## Instantiating our quantum simulator at the backend

In [20]:
dev = qml.device("default.qubit", wires=n_qubits)

## Defining the hidden quantum layer which will be repeated num_layers times i.e. 6 times

In [21]:
@qml.qnode(dev)
def quantum_net(weights, x=None):
    """
    The variational quantum circuit.
    """
    AmplitudeEmbedding(x, wires=range(n_qubits),normalize=True, pad=0.3)
    # AngleEmbedding(x, wires=range(n_qubits))
    StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return tuple(qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits))

## Representation of a single layered QNN

## Defining the cost function of the classifier i.e. categorical crossentropy


In [22]:
def softmax(X):
    prob = np.exp(X)
    probs = prob / np.sum(prob, axis=1, keepdims=True)
    return probs
def cost(thetas, X, actual_labels):
    b = X.shape[0]
    yhats = []
    for i in range(b):
        yhat = quantum_net(thetas[0], x=X[i])
        # print(quantum_net.draw())
        yhats.append(yhat)
    st = np.stack(yhats)
    st_prob = softmax(st)
    y = np.zeros((actual_labels.shape[0], n_qubits))
    y[np.arange(actual_labels.shape[0]), actual_labels] = 1
    return -np.mean(y*np.log(st_prob))

## Implementing the training procedure

### 1. Loading the dataset

In [23]:
num_features = 4
data = np.loadtxt("multiclass_classification/iris.csv", delimiter=",")
X = data[:, 0:num_features]
X_pad = X
print("First X sample (padded)    :", X_pad[0])

First X sample (padded)    : [5.1 3.5 1.4 0.2]


### 2. Normalizing the dataset and creating training and validation(test) set

In [24]:
# normalize each input
normalization = np.sqrt(np.sum(X_pad ** 2, -1))
X_norm = (X_pad.T / normalization).T
print("First X sample (normalized):", X_norm[0])

Y = data[:, -1].astype(int)

np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = X_pad[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = X_pad[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X_pad[index[:num_train]]
X_val = X_pad[index[num_train:]]

First X sample (normalized): [0.80377277 0.55160877 0.22064351 0.0315205 ]


### 3. Initializing the weights and optimization routing for the quantum neural network

In [25]:
theta_weights = strong_ent_layers_uniform(num_layers, n_qubits, seed=42)
theta_bias = 0.0
theta_init = (theta_weights, theta_bias)  # initial weights
opt = AdamOptimizer(l_r)
batch_size = 10
var = theta_init

### 4. Full Circuit Representation

In [26]:
yhat = quantum_net(theta_init[0], x=X_train[0])
print(quantum_net.draw())

 0: ──╭QubitStateVector(M0)──Rot(2.353, 5.974, 4.599)──╭C───────────────────────────────╭X──Rot(4.449, 0.129, 6.094)──╭C──╭X───Rot(2.714, 1.83, 3.844)────────────────────────────╭C──────╭X──┤ ⟨Z⟩ 
 1: ──├QubitStateVector(M0)──Rot(3.761, 0.98, 0.98)────╰X──╭C──Rot(5.23, 1.334, 1.142)──│─────────────────────────────│───╰C──╭X────────────────────────Rot(0.876, 1.836, 2.302)──╰X──╭C──│───┤ ⟨Z⟩ 
 2: ──╰QubitStateVector(M0)──Rot(0.365, 5.442, 3.777)──────╰X───────────────────────────╰C──Rot(1.152, 1.912, 3.297)──╰X──────╰C────────────────────────Rot(2.866, 4.933, 1.255)──────╰X──╰C──┤ ⟨Z⟩ 
M0 =
[0.67767924 0.32715549 0.59589036 0.28041899 0.         0.
 0.         0.        ]



### 5. Initiating the training loop

In [27]:
for it in range(num_epochs):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    var = opt.step(lambda v: cost(v, feats_train_batch, Y_train_batch), var)
    predictions_train = [quantum_net(var[0], x=f) for f in feats_train]
    predictions_val = [quantum_net(var[0], x=f) for f in feats_val]
    pred_train_labels = np.argmax(softmax(predictions_train), axis=1)
    pred_val_labels = np.argmax(softmax(predictions_val), axis=1)
    # Compute accuracy on train and validation set
    acc_train = np.mean(Y_train==pred_train_labels)
    acc_val = np.mean(Y_val==pred_val_labels)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(var, X, Y), acc_train, acc_val)
    )


Iter:     1 | Cost: 0.4173772 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:     2 | Cost: 0.4094382 | Acc train: 0.1339286 | Acc validation: 0.1578947 
Iter:     3 | Cost: 0.4040983 | Acc train: 0.0000000 | Acc validation: 0.0000000 
Iter:     4 | Cost: 0.3968759 | Acc train: 0.0357143 | Acc validation: 0.0526316 
Iter:     5 | Cost: 0.3883290 | Acc train: 0.3303571 | Acc validation: 0.2894737 
Iter:     6 | Cost: 0.3812655 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:     7 | Cost: 0.3744763 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:     8 | Cost: 0.3678104 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:     9 | Cost: 0.3612850 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:    10 | Cost: 0.3557522 | Acc train: 0.3392857 | Acc validation: 0.3157895 
Iter:    11 | Cost: 0.3516366 | Acc train: 0.4375000 | Acc validation: 0.3947368 
Iter:    12 | Cost: 0.3496112 | Acc train: 0.4553571 | Acc validation: 0.4210526 
Iter:    13 | Co